# Review of Misclassified Predictions

In this notebook, we...

## Import packages

In [1]:
import pandas as pd
import pickle

In [2]:


path = '../data/exported_data/'

filename = 'results_df_notebook2.pkl'
openfile = open(path+filename, "rb")
results_df = pickle.load(openfile)
openfile.close()

filename = 'full_output_dict_notebook2.pkl'
openfile = open(path+filename, "rb")
full_output_dict = pickle.load(openfile)
openfile.close()



In [3]:
def get_reshaped_lemmatized():
    # Import the CSV file
    df = pd.read_csv('../data/reshaped_lemmatized.csv')
    return df
df = get_reshaped_lemmatized()

In [4]:
full_output_dict

[{'target': 'nontrigger',
  'model': 'lr',
  'metric': 'roc_auc',
  'model_input': 'SentenceLemmas',
  'split_data': {'X_train': 2150     trigger period mean a period a commence upon ...
   547      low cash flow period mean a period a commence...
   2165    after a trigger period have end any fund avail...
   669     the borrower must pay to the mortgage lender o...
   528     the borrower be require to pay or because to b...
                                 ...                        
   567     borrower be entitle to at borrower s sole opti...
   987     the borrower be not permit to make a request f...
   2183    as a result the lender s control over these re...
   52      upon the occurrence and during the continuance...
   1577    see description of the mortgage loan lockbox a...
   Name: SentenceLemmas, Length: 1596, dtype: object,
   'X_test': 281     operate expense reserve on each payment date o...
   1469           see loan term herein for more information 
   1967    from a

In [5]:
import pandas as pd

In [6]:
# To allow us to better read the Sentences
pd.set_option('display.max_colwidth', None)

In [7]:
pd.read_csv(f'../data/exported_data/results_df_fulldata.csv')

,Unnamed: 0,target,model,metric,model_input,split_data,model_result,best_params,count_deduplicated,test_data_count,best_crossval_score,train_score,test_score
0,0,nontrigger,lr,roc_auc,SentenceLemmas,"{'X_train': 2150 trigger period mean a period a commence upon ...\n547 low cash flow period mean a period a commence...\n2165 after a trigger period have end any fund avail...\n669 the borrower must pay to the mortgage lender o...\n528 the borrower be require to pay or because to b...\n ... \n567 borrower be entitle to at borrower s sole opti...\n987 the borrower be not permit to make a request f...\n2183 as a result the lender s control over these re...\n52 upon the occurrence and during the continuance...\n1577 see description of the mortgage loan lockbox a...\nName: SentenceLemmas, Length: 1596, dtype: object, 'X_test': 281 operate expense reserve on each payment date o...\n1469 see loan term herein for more information \n1967 from and after the first date that the fund on...\n2107 provide no mortgage loan event of default exis...\n1994 not later than ten 10 business day follow deli...\n ... \n1450 approve operating expense mean an operate exp...\n1965 if a trigger period have occur and be continue...\n2222 however the inability of the borrower to compl...\n1085 if quarterly financial statement have not yet ...\n1105 the lockbox account be each own by a separate ...\nName: SentenceLemmas, Length: 684, dtype: object, 'y_train': 2150 0\n547 0\n2165 0\n669 0\n528 0\n ..\n567 0\n987 1\n2183 0\n52 0\n1577 1\nName: nontrigger, Length: 1596, dtype: int64, 'y_test': 281 0\n1469 1\n1967 0\n2107 0\n1994 0\n ..\n1450 1\n1965 0\n2222 0\n1085 1\n1105 1\nName: nontrigger, Length: 684, dtype: int64, 'indices_train': Int64Index([2150, 547, 2165, 669, 528, 1438, 1747, 877, 113, 1258,\n ...\n 1939, 1774, 305, 880, 519, 567, 987, 2183, 52, 1577],\n dtype='int64', length=1596), 'indices_test': Int64Index([ 281, 1469, 1967, 2107, 1994, 1566, 863, 2223, 1275, 134,\n ...\n 1031, 446, 1463, 449, 2196, 1450, 1965, 2222, 1085, 1105],\n dtype='int64', length=684)}","GridSearchCV(cv=3, error_score=nan,\n estimator=Pipeline(memory=None,\n steps=[('cvec',\n CountVectorizer(analyzer='word',\n binary=False,\n decode_error='strict',\n dtype=<class 'numpy.int64'>,\n encoding='utf-8',\n input='content',\n lowercase=True,\n max_df=1.0,\n max_features=None,\n min_df=1,\n ngram_range=(1, 1),\n preprocessor=None,\n stop_words=None,\n strip_accents=None,\n token_pattern='(?u)...\n 'be', 'for', 'with', 'that',\n 'marriott'],\n ['something', 'whereafter', 'are',\n 'full', 'of', 'neither', 'since',\n 'sixty', 'twelve', 'here', 'it',\n 'she', 'a', 'do', 'everyone',\n 'this', 'bottom', 'used',\n 'anywhere', 'never', 'the',\n 'mine', 'thru', 'either', 'give',\n 'serious', 'more', 'my', 'seems',\n 'just', ...]]},\n pre_dispatch='2*n_jobs', refit=True, return_train_score=False,\n scoring='roc_auc', verbose=0)","{'cvec__max_df': 0.99, 'cvec__max_features': 1000, 'cvec__min_df': 2, 'cvec__ngram_range': (1, 3), 'cvec__stop_words': ['something', 'whereafter', 'are', 'full', 'of', 'neither', 'since', 'sixty', 'twelve', 'here', 'it', 'she', 'a', 'do', 'everyone', 'this', 'bottom', 'used', 'anywhere', 'never', 'the', 'mine', 'thru', 'either', 'give', 'serious', 'more', 'my', 'seems', 'just', 'i', 'formerly', 'namely', 'through', 'why', 'empty', 'yet', 'mostly', 'if', 'they', 'will', 'sometime', 'without', 'amount', 'get', 'has', 'nine', 'quite', 'does', 'though', 'whereupon', 'very', 'sometimes', 'whose', 'eleven', 'otherwise', 'side', 'made', 'me', 'hence', 'however', 'next', 'nor', 'whatever', 'third', 'themselves', 'seeming', 'him', 'back', 'did', 'herself', 'on', 'afterwards', 'their', 'thereupon', 'few', 'further', 'under', 'nobody', 'what', 'may', 'myself', 'then', 'again', 'around', 'that', 'after', 'keep', 'as', 'how', 'therein', 'behind', 'across', 're', 'down', 'really', 'alone', 'most', 'being', 'into', 'nowhere', 'among', 'its', 'ours', 'part', 'per', 'almost', 'below',

## Define functions

In [8]:
def get_modeling_results():
    # Import the CSV file
    results_df = pd.read_csv(f'../data/exported_data/results_df_fulldata.csv')
    return results_df

In [9]:
def create_summary_result_table(results_df, df):
    
#    results_df = get_modeling_results()
    
    global final_misclassified_as_true
    global final_misclassified_as_false
    
    target_list = results_df['target'].values.tolist()
    final_columns = ['prediction', 'actual'] + target_list
    
    final_misclassified_as_true = pd.DataFrame(columns = final_columns)
    final_misclassified_as_false = pd.DataFrame(columns = final_columns)

    for row in results_df.itertuples(index = True, name = 'Pandas'):
        best_model = getattr(row, "model_result").best_estimator_
        preds = best_model.predict(getattr(row, "split_data")['X_test'])
        
        index_list = list(getattr(row, "split_data")['indices_test'])
        preds_list = list(preds.astype(int))
        actuals_list = list(getattr(row, "split_data")['y_test'].astype(int))
        
        results = pd.DataFrame({'index': index_list, 'prediction': preds_list, 'actual': actuals_list})
        results.set_index('index', inplace = True)
        
        misclassified = results[results['prediction'] != results['actual']]
        misclassified = misclassified.merge(df, how = 'left', left_index = True, right_index = True)
        misclassified_as_true = misclassified.loc[misclassified['prediction'] == 1]
        misclassified_as_false = misclassified.loc[misclassified['prediction'] == 0]
        
        misclassified_as_true_summary = misclassified_as_true.drop(columns = ['Document', 'Sentence', 'SentenceTokens', 'SentenceLemmas'])
        misclassified_as_false_summary = misclassified_as_false.drop(columns = ['Document', 'Sentence', 'SentenceTokens', 'SentenceLemmas'])
        
        
        misclassified_as_true_summary.loc[f'{getattr(row, "target")}',:] = misclassified_as_true.sum(axis = 0).copy(deep = True)
        misclassified_as_false_summary.loc[f'{getattr(row, "target")}',:] = misclassified_as_false.sum(axis = 0).copy(deep = True)

        
        final_misclassified_as_true = final_misclassified_as_true.append(misclassified_as_true_summary.iloc[-1], ignore_index = False)
        final_misclassified_as_false = final_misclassified_as_false.append(misclassified_as_false_summary.iloc[-1], ignore_index = False)
        
    return final_misclassified_as_true
    return final_misclassified_as_false
        
 

In [10]:
create_summary_result_table(results_df, df)

,prediction,actual,nontrigger,loan_default,unspecified,debt_yield_fall,mezzanine_default,bankruptcy,tenant_failure,renovations,aggregate_debt_yield_fall,dscr_fall,operator_termination,sponsor_termination,sff,mezzanine_outstanding,aggregate_dscr_fall,delayed_repayment,Unnamed: 0
nontrigger,7.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,8555.0
loan_default,9.0,0.0,2.0,0.0,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15125.0
unspecified,17.0,0.0,6.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,13411.0
debt_yield_fall,3.0,0.0,0.0,3.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1510.0
mezzanine_default,6.0,0.0,1.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9903.0
bankruptcy,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,122.0
tenant_failure,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
renovations,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aggregate_debt_yield_fall,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
dscr_fall,3.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,6116.0


In [11]:
results_df.set_index('target', inplace = True)

In [12]:
final_misclassified_as_true

,prediction,actual,nontrigger,loan_default,unspecified,debt_yield_fall,mezzanine_default,bankruptcy,tenant_failure,renovations,aggregate_debt_yield_fall,dscr_fall,operator_termination,sponsor_termination,sff,mezzanine_outstanding,aggregate_dscr_fall,delayed_repayment,Unnamed: 0
nontrigger,7.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,8555.0
loan_default,9.0,0.0,2.0,0.0,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15125.0
unspecified,17.0,0.0,6.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,13411.0
debt_yield_fall,3.0,0.0,0.0,3.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1510.0
mezzanine_default,6.0,0.0,1.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9903.0
bankruptcy,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,122.0
tenant_failure,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
renovations,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aggregate_debt_yield_fall,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
dscr_fall,3.0,0.0,0.0,2.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,6116.0


In [13]:
final_misclassified_as_false = results_df[['count_deduplicated', 'train_score', 'test_score']].merge(final_misclassified_as_false, left_index = True, right_index = True)
final_misclassified_as_false

,count_deduplicated,train_score,test_score,prediction,actual,nontrigger,loan_default,unspecified,debt_yield_fall,mezzanine_default,...,renovations,aggregate_debt_yield_fall,dscr_fall,operator_termination,sponsor_termination,sff,mezzanine_outstanding,aggregate_dscr_fall,delayed_repayment,Unnamed: 0
nontrigger,1030,0.999823,0.985937,0.0,11.0,11.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13407.0
loan_default,553,0.999221,0.991220,0.0,10.0,0.0,10.0,0.0,4.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,9288.0
unspecified,498,0.999828,0.964248,0.0,16.0,0.0,0.0,16.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,24135.0
debt_yield_fall,188,0.999959,0.997085,0.0,4.0,0.0,3.0,0.0,4.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3247.0
mezzanine_default,72,1.000000,0.994850,0.0,1.0,0.0,1.0,0.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1811.0
bankruptcy,44,0.999897,0.995529,0.0,3.0,0.0,3.0,0.0,3.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2310.0
tenant_failure,74,1.000000,0.999863,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2096.0
renovations,26,0.888889,0.933333,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2278.0
aggregate_debt_yield_fall,19,1.000000,0.800000,0.0,2.0,0.0,1.0,0.0,0.0,1.0,...,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1127.0
dscr_fall,23,1.000000,0.750000,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0


In [14]:
final_misclassified_as_true = results_df[['count_deduplicated', 'test_data_count','train_score', 'test_score']].merge(final_misclassified_as_true, left_index = True, right_index = True)
final_misclassified_as_true

,count_deduplicated,test_data_count,train_score,test_score,prediction,actual,nontrigger,loan_default,unspecified,debt_yield_fall,...,renovations,aggregate_debt_yield_fall,dscr_fall,operator_termination,sponsor_termination,sff,mezzanine_outstanding,aggregate_dscr_fall,delayed_repayment,Unnamed: 0
nontrigger,1030,309,0.999823,0.985937,7.0,0.0,0.0,1.0,0.0,0.0,...,2.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,8555.0
loan_default,553,166,0.999221,0.991220,9.0,0.0,2.0,0.0,4.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15125.0
unspecified,498,149,0.999828,0.964248,17.0,0.0,6.0,7.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,2.0,0.0,0.0,13411.0
debt_yield_fall,188,56,0.999959,0.997085,3.0,0.0,0.0,3.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1510.0
mezzanine_default,72,22,1.000000,0.994850,6.0,0.0,1.0,3.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9903.0
bankruptcy,44,13,0.999897,0.995529,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,122.0
tenant_failure,74,22,1.000000,0.999863,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
renovations,26,8,0.888889,0.933333,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
aggregate_debt_yield_fall,19,6,1.000000,0.800000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
dscr_fall,23,7,1.000000,0.750000,3.0,0.0,0.0,2.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,6116.0


In [15]:
results_df.reset_index(inplace = True)

In [16]:
# Export to CSV file

#final_misclassified_as_false.to_csv(f'../data/exported_data/misclassified_as_false_06_16_20.csv')
#final_misclassified_as_true.to_csv(f'../data/exported_data/misclassified_as_true_06_16_20.csv')

In [17]:
results_df.reset_index(inplace = True)